In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import math

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
FILL_IN = "FILL_IN"

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-05-12 03:56:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.4’

smsspamcollection.z     [   <=>              ] 198.65K   307KB/s    in 0.6s    

2025-05-12 03:56:07 (307 KB/s) - ‘smsspamcollection.zip.4’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-05-12 03:56:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.5’

smsspamcollection.z     [   <=>              ] 198.65K   309KB/s    in 0.6s    

2025-05-12 03:56:09 (309 KB/s) - ‘smsspamcollection.zip.5’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = df.text.values
labels = df.label.values

In [ ]:
# Get the bert-base-uncased tokenizer and set lower case to True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ hm       │       20287 │
├──────────┼─────────────┤
│ ##ph     │        8458 │
├──────────┼─────────────┤
│ .        │        1012 │
├──────────┼─────────────┤
│ go       │        2175 │
├──────────┼─────────────┤
│ head     │        2132 │
├──────────┼─────────────┤
│ ,        │        1010 │
├──────────┼─────────────┤
│ big      │        2502 │
├──────────┼─────────────┤
│ ball     │        3608 │
├──────────┼─────────────┤
│ ##er     │        2121 │
├──────────┼─────────────┤
│ .        │        1012 │
╘══════════╧═════════════╛


In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
        input_text,
        max_length=32,
        truncation=True,
        padding='max_length',
        return_tensors='pt',
        return_attention_mask=True,
        return_token_type_ids=False
    )

for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels =  torch.tensor(labels)

In [ ]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

╒════════════╤═════════════╤══════════════════╕
│ Tokens     │   Token IDs │   Attention Mask │
╞════════════╪═════════════╪══════════════════╡
│ [CLS]      │         101 │                1 │
├────────────┼─────────────┼──────────────────┤
│ no         │        2053 │                1 │
├────────────┼─────────────┼──────────────────┤
│ my         │        2026 │                1 │
├────────────┼─────────────┼──────────────────┤
│ blankets   │       15019 │                1 │
├────────────┼─────────────┼──────────────────┤
│ are        │        2024 │                1 │
├────────────┼─────────────┼──────────────────┤
│ sufficient │        7182 │                1 │
├────────────┼─────────────┼──────────────────┤
│ ,          │        1010 │                1 │
├────────────┼─────────────┼──────────────────┤
│ th         │       16215 │                1 │
├────────────┼─────────────┼──────────────────┤
│ ##x        │        2595 │                1 │
├────────────┼─────────────┼────────────

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(np.arange(len(labels)), test_size=val_ratio, stratify=labels, random_state=42)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])

val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(train_set, sampler=RandomSampler(train_set), batch_size=batch_size)

validation_dataloader = DataLoader(val_set, sampler=SequentialSampler(val_set), batch_size=batch_size)

Define the LoRA specific layers.

In [ ]:
# Define a LoRA Layer which has A, B and alpha parameters
class LoRALayer(torch.nn.Module):
  def __init__(self, in_dim, out_dim, rank, alpha):
    super().__init__()
    # Initialize A to be a parameter matrix of dimension in_dim by rank
    self.A = nn.Parameter(torch.randn(in_dim, rank))
    # Initialize all the elements of A via kaiming_uniform with a equal to sqrt(5)
    nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
    # Initialize B to be a zero parameter matrix of the appropriate dimensions
    self.B = nn.Parameter(torch.zeros(rank, out_dim))
    self.alpha = alpha
    self.rank = rank

  def forward(self, x):
    # Pass x through the LoRA layer and return the new x
       x = (x @ self.A @ self.B)*self.alpha/self.rank
       return x

# Define a class LoRALinear which has a linear layer and a LoRA layer on top
class LoRALinear(torch.nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(linear.in_features, linear.out_features, rank, alpha)

  def forward(self, x):
    # Pass x through the linear layer and also the lora layer
    return self.linear(x) + self.lora(x)

def lora_linear_replace(model, rank, alpha):
  # Use model.named_children to go through all layers and if the layer is Linear replace that layer with LoRALinear
  for name, module in model.named_children():
    # If the module is linear, replace the module in the model with a LoRA layer
    if isinstance(module, nn.Linear):
      setattr(model, name, LoRALinear(module, rank, alpha))
    else:
      # Alterntively, recursively apply the same function to child modules so that other Linear layers get replaced
      lora_linear_replace(module, rank, alpha)

### Load specific versions of the model

In [ ]:
import torch.nn as nn

# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

model = model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
    )

# Turn off all gradients of the model to start
for param in model.parameters():
    param.requires_grad = False

# Set to True if LORA is used; if False, fine_tune flag will be used to decide if you fine tune the entire model or just parts
use_lora  = False
# If this is False, turn off gradients
fine_tune = False
# Set total_parameters to 0; this will count the number of parameters in each case
total_parameters = 0

if use_lora:
  # Use the lora_linear to attach a LoRA layer to each linear later of the original BERT model
  lora_linear_replace(model, rank=8, alpha=16)
  # Get the total number of parameters with gradients
  total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

else:
  # If fine_tune is off, turn off gradients for all layers other than classifier
  if not fine_tune:
  # Turn off all gradients; count just the 'classifier' layer which is the only one that has gradients
      for name, param in model.named_parameters():
          if 'classifier' in name:
              param.requires_grad = True
          else:
              param.requires_grad = False
      total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
  else:
    for name, param in model.named_parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(total_parameters)

if use_lora:
  assert(total_parameters == 1345552)
else:
  if fine_tune:
    assert(total_parameters == 109483778)
  else:
    assert(total_parameters == 1538)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1538


### Set the model to the right device

In [ ]:
import platform

# Pick the system you have and select GPU if you can
if platform.system() == 'Darwin':
    device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
elif platform.system() == 'Linux':
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
else:
    device = torch.device('cpu')
print(device)

cuda


In [ ]:
_ = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [ ]:
# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 3e-5,
    eps = 1e-08
)

### Train the model

In [ ]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task="binary")
recall = Recall(task="binary")
precision = Precision(task="binary")
auroc = AUROC(task="binary")

In [ ]:
model.device
epochs = 2

In [ ]:
# Main training / validation loop
import tqdm

for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = [t.to(device) for t in batch]
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set the gradients to zero
        model.zero_grad()

        # Forward pass
        train_output = model(
            input_ids=b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = train_output.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = [t.to(device) for t in batch]
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
            eval_output = model(
                input_ids=b_input_ids,
                attention_mask=b_input_mask
            )
            logits = eval_output.logits

        # Calculate validation metrics
        labels = b_labels.detach().cpu()
        predicted_labels = torch.argmax(logits, dim=1).detach().cpu()
        probs = torch.softmax(logits, dim=1)[:, 1].detach().cpu()

        val_accuracy.append(accuracy(predicted_labels, labels).item())
        val_recall.append(recall(predicted_labels, labels).item())
        val_precision.append(precision(predicted_labels, labels).item())
        val_auroc.append(auroc(probs, labels).item())

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)

Epoch:  50%|█████     | 1/2 [00:23<00:23, 23.21s/it]


	 - Train loss: 0.1469
	 - Validation Accuracy: 0.9866
	 - Validation Precision: 0.8583
	 - Validation Recall: 0.8488
	 - Validation AUROC: 0.9118




100%|██████████| 2/2 [00:45<00:00, 22.86s/it]


	 - Train loss: 0.0366
	 - Validation Accuracy: 0.9857
	 - Validation Precision: 0.8488
	 - Validation Recall: 0.7983
	 - Validation AUROC: 0.9122



### Test on a specific sentence, see the outcome

In [ ]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Ham


### Questions

Question 1: Run the above by fine tuning bert and the classfier head and by not doing this (using BERT as a feature encoder). What is the gap between this?

In [ ]:
# If Fine Tune
import tqdm

for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = [t.to(device) for t in batch]
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set the gradients to zero
        model.zero_grad()

        # Forward pass
        train_output = model(
            input_ids=b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = train_output.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = [t.to(device) for t in batch]
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
            eval_output = model(
                input_ids=b_input_ids,
                attention_mask=b_input_mask
            )
            logits = eval_output.logits

        # Calculate validation metrics
        labels = b_labels.detach().cpu()
        predicted_labels = torch.argmax(logits, dim=1).detach().cpu()
        probs = torch.softmax(logits, dim=1)[:, 1].detach().cpu()

        val_accuracy.append(accuracy(predicted_labels, labels).item())
        val_recall.append(recall(predicted_labels, labels).item())
        val_precision.append(precision(predicted_labels, labels).item())
        val_auroc.append(auroc(probs, labels).item())

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)

Epoch:  50%|█████     | 1/2 [00:32<00:32, 32.55s/it]


	 - Train loss: 0.0795
	 - Validation Accuracy: 0.9821
	 - Validation Precision: 0.8400
	 - Validation Recall: 0.8524
	 - Validation AUROC: 0.9086




100%|██████████| 2/2 [01:05<00:00, 32.84s/it]


	 - Train loss: 0.0288
	 - Validation Accuracy: 0.9884
	 - Validation Precision: 0.8781
	 - Validation Recall: 0.8440
	 - Validation AUROC: 0.9114



In [ ]:
# If nothing
import tqdm

for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = [t.to(device) for t in batch]
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set the gradients to zero
        model.zero_grad()

        # Forward pass
        train_output = model(
            input_ids=b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = train_output.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = [t.to(device) for t in batch]
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
            eval_output = model(
                input_ids=b_input_ids,
                attention_mask=b_input_mask
            )
            logits = eval_output.logits

        # Calculate validation metrics
        labels = b_labels.detach().cpu()
        predicted_labels = torch.argmax(logits, dim=1).detach().cpu()
        probs = torch.softmax(logits, dim=1)[:, 1].detach().cpu()

        val_accuracy.append(accuracy(predicted_labels, labels).item())
        val_recall.append(recall(predicted_labels, labels).item())
        val_precision.append(precision(predicted_labels, labels).item())
        val_auroc.append(auroc(probs, labels).item())

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:  50%|█████     | 1/2 [00:09<00:09,  9.88s/it]


	 - Train loss: 0.4351
	 - Validation Accuracy: 0.8648
	 - Validation Precision: 0.0000
	 - Validation Recall: 0.0000
	 - Validation AUROC: 0.6092




100%|██████████| 2/2 [00:20<00:00, 10.31s/it]


	 - Train loss: 0.3818
	 - Validation Accuracy: 0.8657
	 - Validation Precision: 0.0000
	 - Validation Recall: 0.0000
	 - Validation AUROC: 0.7892



When neither LoRA is used nor BERT is fine-tuned, the model updates only the classifier head, totaling just 1,538 trainable parameters. This configuration trains extremely fast due to the small number of parameters. However, performance is significantly limited, with a validation accuracy of only 86.66%, reflecting the model’s inability to adapt BERT’s internal representations.

In contrast, when we fully fine-tune BERT, the number of trainable parameters jumps to 109,483,778, encompassing the entire model. While this results in a much longer training time, even for only 2 epochs, the performance improves substantially, achieving a validation accuracy of 98.84%.

Using LoRA offers a strong trade-off. With just 1,345,552 trainable parameters, roughly 1% of full fine-tuning, we obtain an accuracy of 98.57%, very close to full fine-tuning with only a 0.3% gap. This demonstrates that LoRA is highly effective in adapting large models efficiently while dramatically reducing computational costs.